In [3]:
import itk

In [45]:
# Median Filtering 
radius=4 #radius of kernel for median filter
dimension=3
InputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/series1_tilescan4_channel3_cropped_sp300_300_ep_600_600.tif'
OutputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/median.mha'
PixelType=itk.F
ImageType=itk.Image[PixelType,dimension]
reader=itk.ImageFileReader[ImageType].New(FileName=InputFileName)

MedianFilter=itk.MedianImageFilter[ImageType,ImageType].New()
MedianFilter.SetInput(reader.GetOutput())
MedianFilter.SetRadius(radius)

writer=itk.ImageFileWriter[ImageType].New()
writer.SetFileName(OutputFileName)
writer.SetInput(MedianFilter.GetOutput())
writer.Update()
# PixelType=itk.UC
# ImageType=itk.Image[PixelType,dimension]
# RescaleFilter=itk.RescaleIntensityImageFilter[Image]

In [58]:
# Multiscale Hessian-Based Measure and Rescaling to [0 255] of Median Image
#Multiscale Hessian-Based Measure
SigmaMin=2
SigmaMax=5
NumSigmaSteps=10
InputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/median.mha'
OutputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/plane.mha'
InputImage=itk.imread(InputFileName,itk.F)
ImageType=type(InputImage)
dimension=3
HessianPixelType=itk.SymmetricSecondRankTensor[itk.D,dimension]
HessianImageType=itk.Image[HessianPixelType,dimension]

ObjectnessFilter=itk.HessianToObjectnessMeasureImageFilter[HessianImageType,ImageType].New()
ObjectnessFilter.SetBrightObject(True)
ObjectnessFilter.SetScaleObjectnessMeasure(True)
ObjectnessFilter.SetAlpha(1.0)
ObjectnessFilter.SetBeta(1.0)
ObjectnessFilter.SetGamma(1.0)
ObjectnessFilter.SetObjectDimension(2) #M=2 for 2D plate-like structures

MultiScaleFilter=itk.MultiScaleHessianBasedMeasureImageFilter[ImageType,HessianImageType, ImageType].New()
MultiScaleFilter.SetInput(InputImage)
MultiScaleFilter.SetHessianToMeasureFilter(ObjectnessFilter)
#MultiScaleFilter.SetSigmaStepToLogarithmic()
MultiScaleFilter.SetSigmaMaximum(SigmaMax)
MultiScaleFilter.SetSigmaMinimum(SigmaMin)
MultiScaleFilter.SetNumberOfSigmaSteps(NumSigmaSteps)

#Rescaling
OutputPixelType=itk.UC
OutputImageType=itk.Image[OutputPixelType,dimension]

RescaleFilter=itk.RescaleIntensityImageFilter[ImageType,OutputImageType].New()
RescaleFilter.SetInput(MultiScaleFilter)

itk.imwrite(RescaleFilter.GetOutput(),OutputFileName)
itk.imwrite(RescaleFilter.GetOutput(),'/home/jwk114/CardiomyocytesSegmentation/Images/plane.tif')

In [53]:
#Watershed of Median Image
InputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/median.mha'
OutputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/watershed.mha'
dimension=3
PixelType=itk.F
ImageType=itk.Image[PixelType,dimension]
reader=itk.ImageFileReader[ImageType].New()
reader.SetFileName(InputFileName)

WatershedFilter=itk.WatershedImageFilter.New(Input=reader.GetOutput())
threshold=0
level=0.2
WatershedFilter.SetThreshold(threshold)
WatershedFilter.SetLevel(level)

LabelledImageType=type(WatershedFilter.GetOutput())
PixelType=itk.UC
RGBPixelType=itk.RGBPixel[PixelType]
RGBImageType=itk.Image[RGBPixelType,dimension]

ScalarToRGBColormapFilterType=\
itk.ScalarToRGBColormapImageFilter[LabelledImageType,RGBImageType]
ColormapImageFilter = ScalarToRGBColormapFilterType.New()
ColormapImageFilter.SetColormap(ScalarToRGBColormapFilterType.Jet)
ColormapImageFilter.SetInput(WatershedFilter.GetOutput())
ColormapImageFilter.Update()

WriterType = itk.ImageFileWriter[RGBImageType]
writer = WriterType.New()
writer.SetFileName(OutputFileName)
writer.SetInput(ColormapImageFilter.GetOutput())
writer.Update()


In [100]:
tree=WatershedFilter.GetSegmentTree()
#WatershedFilter.GetBasicSegmentation()
# help(tree)
itk.WatershedS

Help on SwigPyObject object:

class SwigPyObject(object)
 |  Swig object carries a C/C++ instance pointer
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __int__(self, /)
 |      int(self)
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  acquire(...)
 |      acquires ownership of the pointer
 |  
 |  append(...)
 |      appends another 'this' object
 |  
 |  disown(...)
 |      releases ownership of the pointer
 |  
 |  next(...)
 |      returns the next 'this' object
 |  
 |  own(...)
 |      returns/sets ownership of the pointer
 |  
 |  -------------

AttributeError: module 'itk' has no attribute 'WatershedSegmentTree'

In [63]:
#Watershed of Morphologically Closed Plane Image
#Not going to work for this implementation of watershed
#because it uses gradient descend to group pixels of the same 
#basin together, but for my thresholded plane image, all we have is
#255 at membranes and 0 at non membranes
InputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/planeClosed_threshold9.mha'
OutputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/watershed_planeClosed_threshold9.mha'
dimension=3
PixelType=itk.F
ImageType=itk.Image[PixelType,dimension]
reader=itk.ImageFileReader[ImageType].New()
reader.SetFileName(InputFileName)

WatershedFilter=itk.WatershedImageFilter.New(Input=reader.GetOutput())
threshold=0
level=0.05
WatershedFilter.SetThreshold(threshold)
WatershedFilter.SetLevel(level)

LabelledImageType=type(WatershedFilter.GetOutput())
PixelType=itk.UC
RGBPixelType=itk.RGBPixel[PixelType]
RGBImageType=itk.Image[RGBPixelType,dimension]

ScalarToRGBColormapFilterType=\
itk.ScalarToRGBColormapImageFilter[LabelledImageType,RGBImageType]
ColormapImageFilter = ScalarToRGBColormapFilterType.New()
ColormapImageFilter.SetColormap(ScalarToRGBColormapFilterType.Jet)
ColormapImageFilter.SetInput(WatershedFilter.GetOutput())
ColormapImageFilter.Update()

WriterType = itk.ImageFileWriter[RGBImageType]
writer = WriterType.New()
writer.SetFileName(OutputFileName)
writer.SetInput(ColormapImageFilter.GetOutput())
writer.Update()